# Week 5: Gradient Boosted Trees и SVM

Цель: изучить теорию и практику обучения моделей Gradient Boosted Trees (XGBoost/LightGBM/CatBoost) и SVM (linear и kernel), а также сравнить их производительность с Random Forest.

## Учебные цели

- Понять формальную постановку Gradient Boosting, включая шаги оптимизации и регуляризацию.
- Освоить работу с XGBoost, LightGBM, CatBoost (и sklearn fallback).
- Разобрать теорию SVM: primal/dual, hinge loss, kernel trick.
- Научиться обучать и сравнивать linear и kernel SVM с Random Forest.
- Освоить интерпретацию метрик (Accuracy, ROC AUC, F1) и визуализацию границ решений.
- Провести интерактивные эксперименты с гиперпараметрами (C, gamma, n_estimators, learning_rate, max_depth).

## Оглавление

1. Подготовка окружения и данных
2. Теория: Gradient Boosting (формулы и интуиция) + ручной пример
3. Практика: обучение Gradient Boosted Trees (XGBoost/LightGBM/CatBoost)
4. Теория: SVM (linear и kernel) + ручной пример
5. Практика: SVM vs Random Forest — сравнение
6. Интерактивные эксперименты (Plotly + ipywidgets)
7. Плюсы/минусы и рекомендации по применению
8. Домашнее задание

In [1]:
# -*- coding: utf-8 -*-
"""
Импорт библиотек и подготовка окружения

Notes
-----
- Проверяем доступность XGBoost, LightGBM, CatBoost. Если недоступны — используем sklearn аналоги.
- Фиксируем seed для воспроизводимости.
- Визуализация через Plotly.
"""
from __future__ import annotations
import importlib
import warnings
import numpy as np
import pandas as pd
from typing import Optional

# Фиксация seed
SEED: int = 42
rng = np.random.default_rng(SEED)

# Тихий режим для некоторых либ
warnings.filterwarnings("ignore")

# Проверка наличия библиотек
has_xgb = importlib.util.find_spec("xgboost") is not None
has_lgbm = importlib.util.find_spec("lightgbm") is not None
has_cat = importlib.util.find_spec("catboost") is not None
has_widgets = importlib.util.find_spec("ipywidgets") is not None

import plotly.express as px
import plotly.graph_objects as go

print({
    "xgboost": has_xgb,
    "lightgbm": has_lgbm,
    "catboost": has_cat,
    "ipywidgets": has_widgets,
})

{'xgboost': False, 'lightgbm': False, 'catboost': True, 'ipywidgets': True}


## Теория: Gradient Boosting — формулировка, интуиция и практические акценты

Коротко (TL;DR)
- Идея: последовательно добавляем слабые модели (обычно деревья), чтобы исправлять ошибки предыдущего ансамбля по направлению антиградиента функции потерь.
- Математика: на шаге m аппроксимируем антиградиент и подбираем базовую модель, минимизирующую квадратичную ошибку на псевдо-остатках; обновляемся с шагом learning_rate.
- Практика: качество/устойчивость даёт комбинация малых шагов (learning_rate), ограниченной сложности деревьев (max_depth) и стахастических приёмов (subsample, colsample).

### Формальная постановка
Имеется выборка ${(x_i, y_i)}_{i=1}^n$, $x_i \in \mathbb{R}^d$, $y_i \in \mathbb{R}$ (регрессия) или $\{-1,+1\}$ (классификация). Ищем ансамбль
$$
F_0(x) = \arg\min_{\gamma} \sum_{i=1}^n L(y_i, \gamma), \quad
F_M(x) = F_{M-1}(x) + \nu \cdot h_M(x), \quad \nu \in (0,1].
$$
На шаге $m$:
$$
r_{im} = -\left. \frac{\partial L(y_i, F(x_i))}{\partial F(x_i)} \right|_{F=F_{m-1}}, \quad
h_m = \arg\min_h \sum_{i=1}^n (r_{im} - h(x_i))^2.
$$
Для бинарной логистической классификации с $L(y, F) = \log(1 + e^{-yF})$ псевдо-остатки:
$$
r_{im} = \frac{y_i}{1 + e^{y_i F_{m-1}(x_i)}}.
$$
Интуиция: $h_m$ учится «догонять» направление наибольшего улучшения качества (антиградиент) в точках обучающей выборки; маленький $\nu$ стабилизирует обучение («shrinkage»).

### Связь с XGBoost/LightGBM/CatBoost
- XGBoost: минимизирует тэййлорово разложение второго порядка с явной регуляризацией листьев. Целевая на шаге $m$ при добавлении дерева $f$:
$$
\mathcal{L}^{(m)} = \sum_{i=1}^n L\big(y_i, F_{m-1}(x_i)+f(x_i)\big) + \Omega(f),\quad
\Omega(f) = \gamma T + \tfrac{1}{2}\lambda \sum_{j=1}^T w_j^2.
$$
Оптимальные веса листьев $w_j$ и прирост от сплита выражаются через суммы градиентов $g_i$ и гессианов $h_i$ по объектам узла.
- LightGBM: histogram-based обучение + GOSS (Gradient-based One-Side Sampling) ускоряют поиск сплитов.
- CatBoost: Ordered Boosting и аккуратная обработка категориальных признаков снижают target leakage и смещения.

### Практические рекомендации
- Базовые гиперпараметры: увеличивайте $n\_estimators$ вместе с уменьшением `learning_rate` (например, 0.03–0.1); ограничивайте `max_depth` (обычно 3–8); используйте `subsample` и `colsample` 0.6–0.9.
- Ранний стоп: при валидации используйте early stopping, особенно в XGBoost/LightGBM.
- Масштаб признаков: для деревьев не критичен; нормализация не обязательна.

### Глоссарий (ключевые термины)
- learning_rate (shrinkage): коэффициент, уменьшающий вклад очередной базовой модели.
- base learner: слабая модель, чаще всего дерево небольшой глубины.
- (псевдо-)остатки: оценка направления антиградиента на объектах.
- subsample/colsample: стохастический подотбор объектов/признаков на шаге.
- регуляризация (XGBoost): $\gamma$ (штраф за листья), $\lambda$ (L2 на весах листьев).

<details>
<summary>Проверка понимания (раскрыть)</summary>
1) Что даёт уменьшение learning_rate при фиксированном числе деревьев? — Более стабильное, но обычно недообученное решение; нужно увеличивать число деревьев.

2) Зачем нужны subsample/colsample? — Вносят стохастичность, снижают вариативность, ускоряют и помогают обобщению.

3) Почему XGBoost использует второй порядок? — Более точная локальная аппроксимация потерь даёт лучшее направление улучшения и явные формулы для весов листьев/прироста сплитов.
</details>

См. далее: «Ручной пример: Gradient Boosting для регрессии (2 итерации)» для пошаговых вычислений на маленьких данных.

### Ручной пример: Gradient Boosting для регрессии (2 итерации)

Рассмотрим простую регрессионную задачу с квадратной функцией потерь $L(y, F) = (y - F)^2/2$. Пусть имеется 4 точки на прямой:

- $x = [0, 1, 2, 3]$, $y = [1, 3, 2, 5]$.
- Базовые модели — decision stumps (пеньки): $h(x) = a$ при $x < t$ и $h(x) = b$ при $x \ge t$.
- Learning rate $\nu = 0.5$.

Шаг 0 (инициализация):
- $F_0(x) = \arg\min_\gamma \sum_i (y_i - \gamma)^2/2 = \bar y = (1+3+2+5)/4 = 2.75$.

Итерация 1:
1. Остатки $r_i = y_i - F_0(x_i)$: $[1-2.75,\ 3-2.75,\ 2-2.75,\ 5-2.75] = [-1.75,\ 0.25,\ -0.75,\ 2.25]$.
2. Подбираем stump $h_1(x)$, минимизируя $\sum_i (r_i - h(x_i))^2$. Рассмотрим пороги $t \in \{0.5,\ 1.5,\ 2.5\}$.
   - Для $t=0.5$: левая группа $\{x=0\}$: среднее остатков $=-1.75$; правая группа $\{1,2,3\}$: среднее $=(0.25-0.75+2.25)/3=0.5833$.
     Ошибка: $( -1.75+1.75)^2 + \sum_{x\ge 1}(r_i-0.5833)^2 = 0 + (0.25-0.5833)^2 + (-0.75-0.5833)^2 + (2.25-0.5833)^2 = 0.1111 + 1.7778 + 2.7778 = 4.6667$.
   - Для $t=1.5$: левая $\{0,1\}$: среднее $=(-1.75+0.25)/2=-0.75$; правая $\{2,3\}$: среднее $=(-0.75+2.25)/2=0.75$.
     Ошибка: $(-1.75+0.75)^2 + (0.25+0.75)^2 + (-0.75-0.75)^2 + (2.25-0.75)^2 = 1.0 + 1.0 + 2.25 + 2.25 = 6.5$.
   - Для $t=2.5$: левая $\{0,1,2\}$: среднее $=(-1.75+0.25-0.75)/3=-0.75$; правая $\{3\}$: среднее $=2.25$.
     Ошибка: $(-1.75+0.75)^2 + (0.25+0.75)^2 + (-0.75+0.75)^2 + (2.25-2.25)^2 = 1.0 + 1.0 + 0 + 0 = 2.0$.

   Лучший порог $t=2.5$, параметры $a=-0.75$, $b=2.25$. То есть $h_1(0)=h_1(1)=h_1(2)=-0.75$, $h_1(3)=2.25$.
3. Обновление: $F_1(x) = F_0(x) + \nu h_1(x)$. Значения:
   - $F_1(0)=2.75 + 0.5\cdot(-0.75)=2.375$;
   - $F_1(1)=2.375$;
   - $F_1(2)=2.375$;
   - $F_1(3)=2.75 + 0.5\cdot 2.25=3.875$.

Итерация 2:
1. Остатки $r_i = y_i - F_1(x_i)$: $[1-2.375,\ 3-2.375,\ 2-2.375,\ 5-3.875] = [-1.375,\ 0.625,\ -0.375,\ 1.125]$.
2. Аналогично подбираем второй stump по $r_i$:
   - Для $t=0.5$: лев. $\{-1.375\}$, прав. $\{0.625,-0.375,1.125\}$, средние $a=-1.375$, $b=0.4583$; ошибка $=0+(0.625-0.4583)^2+(-0.375-0.4583)^2+(1.125-0.4583)^2=0.0278+0.6944+0.4444=1.1667$.
   - Для $t=1.5$: лев. $\{-1.375,0.625\}$, прав. $\{-0.375,1.125\}$, средние $a=-0.375$, $b=0.375$; ошибка $=(-1.375+0.375)^2 + (0.625+0.375)^2 + (-0.375-0.375)^2 + (1.125-0.375)^2 = 1.0 + 1.0 + 0.5625 + 0.5625 = 3.125$.
   - Для $t=2.5$: лев. $\{-1.375,0.625,-0.375\}$, прав. $\{1.125\}$, средние $a=-0.375$, $b=1.125$; ошибка $=(-1.375+0.375)^2 + (0.625+0.375)^2 + (-0.375+0.375)^2 + (1.125-1.125)^2 = 1.0 + 1.0 + 0 + 0 = 2.0$.

   Лучший порог снова $t=0.5$ с меньшей ошибкой $1.1667$. Получаем $h_2(0)=-1.375$, $h_2(1)=h_2(2)=h_2(3)=0.4583$.
3. Обновление: $F_2(x) = F_1(x) + 0.5\cdot h_2(x)$:
   - $F_2(0)=2.375 + 0.5\cdot(-1.375)=1.6875$;
   - $F_2(1)=2.375 + 0.5\cdot 0.4583=2.6042$;
   - $F_2(2)=2.6042$;
   - $F_2(3)=3.875 + 0.5\cdot 0.4583=4.1042$.

Замечание: при небольшом learning_rate и большем числе итераций модель продолжит улучшаться. Регуляризация через глубину пеньков/деревьев и subsampling позволяет избегать переобучения.

In [2]:
"""
Генерация синтетических данных для бинарной классификации

Returns
-------
X : ndarray of shape (n_samples, 2)
    Признаки.
y : ndarray of shape (n_samples,)
    Метки классов {-1, +1}.
"""
from sklearn.datasets import make_moons
from sklearn.model_selection import train_test_split

X, y01 = make_moons(n_samples=600, noise=0.25, random_state=SEED)
# Переведём в {-1, +1}
y = np.where(y01 == 1, 1, -1)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=SEED, stratify=y)
X_train.shape, X_test.shape

((420, 2), (180, 2))

In [3]:
"""
Обучение Gradient Boosted Trees: XGBoost/LightGBM/CatBoost или sklearn fallback

Notes
-----
- Для XGBoost/LightGBM/CatBoost используем соответствующие API, если они доступны.
- В качестве запасного варианта используем sklearn.ensemble.GradientBoostingClassifier.
- Оцениваем Accuracy и ROC AUC.
"""
from sklearn.metrics import accuracy_score, roc_auc_score
from sklearn.ensemble import GradientBoostingClassifier

models = {}

if has_xgb:
    from xgboost import XGBClassifier
    models["XGBoost"] = XGBClassifier(
        n_estimators=300, learning_rate=0.1, max_depth=3,
        subsample=0.8, colsample_bytree=0.8, reg_lambda=1.0,
        random_state=SEED, eval_metric="logloss", tree_method="hist"
    )
if has_lgbm:
    from lightgbm import LGBMClassifier
    models["LightGBM"] = LGBMClassifier(
        n_estimators=300, learning_rate=0.1, max_depth=-1,
        num_leaves=31, subsample=0.8, colsample_bytree=0.8,
        reg_lambda=0.0, random_state=SEED
    )
if has_cat:
    from catboost import CatBoostClassifier
    models["CatBoost"] = CatBoostClassifier(
        iterations=300, learning_rate=0.1, depth=6,
        loss_function="Logloss", verbose=False, random_seed=SEED
    )

# Fallback
models["SklearnGB"] = GradientBoostingClassifier(
    n_estimators=300, learning_rate=0.1, max_depth=3, random_state=SEED
)

results = []
for name, clf in models.items():
    clf.fit(X_train, y_train)
    y_pred = clf.predict(X_test)
    # для ROC AUC нужны вероятности класса +1
    if hasattr(clf, "predict_proba"):
        y_proba = clf.predict_proba(X_test)[:, 1]
    elif hasattr(clf, "decision_function"):
        # приводим к [0,1] через сигмоиду для сопоставимости
        z = clf.decision_function(X_test)
        y_proba = 1 / (1 + np.exp(-z))
    else:
        y_proba = (y_pred == 1).astype(float)

    acc = accuracy_score(y_test, y_pred)
    auc = roc_auc_score((y_test==1).astype(int), y_proba)
    results.append((name, acc, auc))

results

[('CatBoost', 0.9333333333333333, 0.975679012345679),
 ('SklearnGB', 0.9166666666666666, 0.9598148148148148)]

In [4]:
"""
Визуализация сравнения моделей Gradient Boosted Trees

Notes
-----
- Используем Plotly для интерактивной таблицы результатов.
- Одна визуализация в ячейке.
"""
res_df = pd.DataFrame(results, columns=["model", "accuracy", "roc_auc"]).sort_values("roc_auc", ascending=False)
fig = go.Figure(data=[
    go.Bar(name="Accuracy", x=res_df["model"], y=res_df["accuracy"], marker_color="#4C78A8"),
    go.Bar(name="ROC AUC", x=res_df["model"], y=res_df["roc_auc"], marker_color="#F58518"),
])
fig.update_layout(barmode="group", title="Сравнение моделей Gradient Boosted Trees", yaxis_title="Score")
fig.show()

## Теория: Support Vector Machines (SVM) — интуиция, формулы и настройки

Коротко (TL;DR)
- Идея: найти гиперплоскость с максимально широким margin, допускающую минимум ошибок; при kernel trick — разделяемость достигается в признаковом пространстве $\phi(x)$.
- Математика: soft-margin минимизирует $\tfrac{1}{2}\|w\|^2 + C\sum\xi_i$ при ограничениях на функциональные значения; дуальная задача приводит к сумме по поддерживающим векторам с ядром $K$.
- Практика: линейный SVM хорош для больших, разреженных признаков; kernel SVM (RBF) — для сложных нелинейных границ на умеренных по размеру данных.

### Примальная и дуальная постановки
Примальная:
$$
\min_{w,b,\xi} \tfrac{1}{2}\|w\|^2 + C\sum_{i=1}^n \xi_i \quad \text{s.t.} \quad y_i(w^\top x_i + b) \ge 1 - \xi_i,\; \xi_i\ge 0.
$$
Дуальная:
$$
\max_{\alpha} \sum_{i=1}^n \alpha_i - \tfrac{1}{2}\sum_{i,j} \alpha_i\alpha_j y_i y_j K(x_i,x_j) \quad \text{s.t.} \quad 0\le \alpha_i \le C,\; \sum_i \alpha_i y_i=0.
$$
Decision function: $f(x)=\sum_i \alpha_i y_i K(x_i,x) + b$. Поддерживающие векторы: объекты с $\alpha_i>0$.

### Kernel trick и RBF ядро
- Kernel trick позволяет работать в высокомерном пространстве $\phi(x)$ без явного отображения признаков.
- Популярное RBF: $K(x,z)=\exp(-\gamma\|x-z\|^2)$. Гиперпараметры: `C` и `gamma` (радиус влияния SV).

### Практические рекомендации
- Масштабируйте признаки (особенно для линейного SVM и SVC(kernel='linear')), иначе C работает непредсказуемо.
- Подбирайте `C` и `gamma` на валидации (лог-шкала: C∈[1e-2,1e2], gamma∈[1e-3,1e0]).
- Для больших, разреженных текстов используйте линейный SVM (LinearSVC/SGDClassifier), для 2D и умеренных данных — SVC(RBF).

### Глоссарий
- margin: расстояние до ближайших точек разметки от разделяющей гиперплоскости.
- hinge loss: $\max(0, 1 - y f(x))$, штрафует за «неправильные» и «недостаточно уверенные» классификации.
- C: вес ошибки в soft-margin; большой C — меньше ошибок на train, выше риск overfitting.
- gamma (RBF): ширина «колокола» ядра; большой gamma — локальные, извилистые границы.

<details>
<summary>Проверка понимания (раскрыть)</summary>
1) Что происходит при очень большом C? — Модель старается исправить все ошибки на обучении, уменьшая margin и повышая риск переобучения.

2) Что происходит при слишком большом gamma (RBF)? — Граница становится чрезмерно извилистой, модель запоминает шум.

3) Почему линейный SVM часто выигрывает на текстах? — Высокая размерность и разреженность делают линейную границу эффективной и быстрой, без необходимости сложных ядер.
</details>

См. далее: «Ручной пример: линейный SVM в 2D» для интуитивной иллюстрации выбора $w,b$ по условиям KKT.

In [5]:
"""
Обучение SVM (линейный и RBF) и Random Forest. Сравнение с Gradient Boosting.

Notes
-----
- Для линейного SVM используем LinearSVC (быстрый для больших d) и/или SVC(kernel='linear').
- Для kernel SVM используем SVC(RBF). Сравним с RandomForestClassifier.
- Считаем Accuracy и ROC AUC.
"""
from sklearn.svm import LinearSVC, SVC
from sklearn.ensemble import RandomForestClassifier

svc_models = {
    "LinearSVC": LinearSVC(C=1.0, random_state=SEED),
    "SVC-linear": SVC(kernel="linear", C=1.0, probability=True, random_state=SEED),
    "SVC-RBF": SVC(kernel="rbf", C=1.0, gamma="scale", probability=True, random_state=SEED),
    "RandomForest": RandomForestClassifier(n_estimators=300, max_depth=None, random_state=SEED)
}

svm_results = []
for name, clf in svc_models.items():
    clf.fit(X_train, y_train)
    y_pred = clf.predict(X_test)
    if hasattr(clf, "predict_proba"):
        y_proba = clf.predict_proba(X_test)[:, 1]
    else:
        z = clf.decision_function(X_test)
        y_proba = 1 / (1 + np.exp(-z))

    acc = accuracy_score(y_test, y_pred)
    auc = roc_auc_score((y_test==1).astype(int), y_proba)
    svm_results.append((name, acc, auc))

pd.DataFrame(svm_results, columns=["model", "accuracy", "roc_auc"]).sort_values("roc_auc", ascending=False)

,model,accuracy,roc_auc
2,SVC-RBF,0.922222,0.981605
3,RandomForest,0.927778,0.970247
0,LinearSVC,0.844444,0.938519
1,SVC-linear,0.855556,0.938519


In [6]:
"""
Визуализация сравнения SVM/RandomForest

Notes
-----
- Строим столбчатую диаграмму для Accuracy и ROC AUC.
- Одна визуализация в ячейке.
"""
svm_df = pd.DataFrame(svm_results, columns=["model", "accuracy", "roc_auc"]).sort_values("roc_auc", ascending=False)
fig = go.Figure(data=[
    go.Bar(name="Accuracy", x=svm_df["model"], y=svm_df["accuracy"], marker_color="#72B7B2"),
    go.Bar(name="ROC AUC", x=svm_df["model"], y=svm_df["roc_auc"], marker_color="#E45756"),
])
fig.update_layout(barmode="group", title="Сравнение SVM/RandomForest", yaxis_title="Score")
fig.show()

In [7]:
"""
Интерактив: визуализация границы решений для 2D данных

Parameters
----------
model_name : str
    Имя модели из словаря, например 'SVC-RBF' или 'XGBoost'.
C : float
    Регуляризация для SVM.
gamma : float or str
    Параметр ядра RBF для SVM. 'scale' или число.
n_estimators : int
    Число деревьев для Boosting.
learning_rate : float
    Learning rate для Boosting.
max_depth : int
    Глубина деревьев для Boosting.
"""
from typing import Optional, Tuple

# Импорты моделей на всякий случай, чтобы ячейка была самодостаточной
from sklearn.svm import SVC, LinearSVC
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier

# Подготовим сетку для отображения
x_min, x_max = X[:,0].min()-0.5, X[:,0].max()+0.5
y_min, y_max = X[:,1].min()-0.5, X[:,1].max()+0.5

xx, yy = np.meshgrid(
    np.linspace(x_min, x_max, 300),
    np.linspace(y_min, y_max, 300)
)

def get_model(model_name: str, C: float=1.0, gamma: Optional[float|str]="scale",
              n_estimators: int=200, learning_rate: float=0.1, max_depth: int=3):
    if model_name == "SVC-RBF":
        return SVC(kernel="rbf", C=C, gamma=gamma, probability=True, random_state=SEED)
    if model_name == "LinearSVC":
        return LinearSVC(C=C, random_state=SEED)
    if model_name == "RandomForest":
        return RandomForestClassifier(n_estimators=200, random_state=SEED)
    # Boosting
    if model_name == "XGBoost" and has_xgb:
        from xgboost import XGBClassifier
        return XGBClassifier(n_estimators=n_estimators, learning_rate=learning_rate,
                             max_depth=max_depth, subsample=0.8, colsample_bytree=0.8,
                             random_state=SEED, eval_metric="logloss", tree_method="hist")
    if model_name == "LightGBM" and has_lgbm:
        from lightgbm import LGBMClassifier
        return LGBMClassifier(n_estimators=n_estimators, learning_rate=learning_rate,
                              max_depth=-1 if max_depth is None else max_depth,
                              num_leaves=31, subsample=0.8, colsample_bytree=0.8,
                              random_state=SEED)
    if model_name == "CatBoost" and has_cat:
        from catboost import CatBoostClassifier
        return CatBoostClassifier(iterations=n_estimators, learning_rate=learning_rate,
                                  depth=max_depth, loss_function="Logloss",
                                  verbose=False, random_seed=SEED)
    # Fallback
    return GradientBoostingClassifier(n_estimators=n_estimators, learning_rate=learning_rate,
                                      max_depth=max_depth, random_state=SEED)


def plot_decision_boundary(model_name: str="SVC-RBF", C: float=1.0, gamma: Optional[float|str]="scale",
                           n_estimators: int=200, learning_rate: float=0.1, max_depth: int=3):
    """
    Построить поверхность вероятности класса +1 и точки обучающей выборки.

    Returns
    -------
    fig : plotly.graph_objects.Figure
        Фигура с контурной картой и точками классов.
    """
    clf = get_model(model_name, C, gamma, n_estimators, learning_rate, max_depth)
    clf.fit(X_train, y_train)

    grid = np.c_[xx.ravel(), yy.ravel()]
    if hasattr(clf, "predict_proba"):
        Z = clf.predict_proba(grid)[:,1]
    else:
        z = clf.decision_function(grid)
        Z = 1/(1+np.exp(-z))
    Z = Z.reshape(xx.shape)

    fig = go.Figure()
    fig.add_trace(go.Contour(x=xx[0], y=yy[:,0], z=Z, colorscale="RdBu", showscale=True,
                             contours=dict(showlines=False)))
    fig.add_trace(go.Scatter(x=X_train[y_train==-1,0], y=X_train[y_train==-1,1], mode="markers",
                             name="-1", marker=dict(color="#1f77b4")))
    fig.add_trace(go.Scatter(x=X_train[y_train== 1,0], y=X_train[y_train== 1,1], mode="markers",
                             name="+1", marker=dict(color="#d62728")))
    fig.update_layout(title=f"Decision boundary: {model_name}")
    return fig

# Включаем интерактив, если ipywidgets доступен. Иначе — статическая демо-визуализация.
if has_widgets:
    import ipywidgets as widgets
    from IPython.display import display, clear_output

    # Доступные модели
    model_options = ["SVC-RBF", "LinearSVC", "RandomForest"]
    if has_xgb:
        model_options.append("XGBoost")
    if has_lgbm:
        model_options.append("LightGBM")
    if has_cat:
        model_options.append("CatBoost")

    # Контролы
    model_dd = widgets.Dropdown(options=model_options, value="SVC-RBF", description="Модель:")
    C_slider = widgets.FloatLogSlider(value=1.0, base=10, min=-2, max=2, step=0.1,
                                      description="C", continuous_update=False)
    gamma_mode = widgets.Dropdown(options=["scale", "float"], value="scale", description="gamma:")
    gamma_slider = widgets.FloatLogSlider(value=0.1, base=10, min=-3, max=0, step=0.1,
                                          description="γ", continuous_update=False)
    n_estimators_slider = widgets.IntSlider(value=200, min=50, max=500, step=50,
                                            description="n_estimators", continuous_update=False)
    lr_slider = widgets.FloatLogSlider(value=0.1, base=10, min=-2, max=0, step=0.1,
                                       description="lr", continuous_update=False)
    depth_slider = widgets.IntSlider(value=3, min=1, max=10, step=1,
                                     description="max_depth", continuous_update=False)

    output = widgets.Output()

    def update(*args):
        with output:
            clear_output(wait=True)
            gamma_arg = "scale" if gamma_mode.value == "scale" else gamma_slider.value
            fig = plot_decision_boundary(model_dd.value,
                                         C=C_slider.value,
                                         gamma=gamma_arg,
                                         n_estimators=n_estimators_slider.value,
                                         learning_rate=lr_slider.value,
                                         max_depth=depth_slider.value)
            fig.show()

    # Подписка на изменения
    for w in [model_dd, C_slider, gamma_mode, gamma_slider, n_estimators_slider, lr_slider, depth_slider]:
        w.observe(lambda change: update(), names="value")

    # Первый рендер
    update()

    # Компоновка UI
    ui = widgets.VBox([
        widgets.HBox([model_dd, C_slider, gamma_mode, gamma_slider]),
        widgets.HBox([n_estimators_slider, lr_slider, depth_slider]),
        output
    ])
    display(ui)
else:
    print("ipywidgets недоступен в этом окружении. Показана статическая демо-визуализация.")
    fig_demo = plot_decision_boundary("SVC-RBF", C=1.0, gamma="scale")
    fig_demo.show()

## Метрики качества: как выбирать и что смотреть в этой задаче

В нашей задаче (бинарная классификация на 2D с умеренным шумом и близко сбалансированными классами) уместно использовать ROC AUC для общего ранжирования моделей, но важно смотреть и на PR AUC, когда положительный класс редок или когда цена ложноположительных/ложноотрицательных велика.

Ключевые метрики:
- Accuracy: доля верных предсказаний. Может вводить в заблуждение при дисбалансе классов.
- Precision/Recall и F1: точность и полнота, их гармоническое среднее. F1 максимизирует баланс между ними при фиксированном пороге.
- ROC AUC: вероятность, что случайный положительный объект получит больший скор, чем случайный отрицательный. Не зависит от одного порога, устойчив к балансировке классов, но скрывает поведение в области высоких требований к Precision/Recall.
- PR AUC (Average Precision): лучше отражает качество при редком положительном классе, фокусируется на Precision-Recall компромиссе.

Выбор метрики под кейс:
- Баланс классов ≈ 50/50 и общая способность модели ранжировать важнее порога — смотрим ROC AUC.
- Редкий положительный класс (например, обнаружение аномалий) — ориентируемся на PR AUC и подбираем порог под нужный trade-off precision/recall.
- Фиксированный порог и чёткая бизнес-цена ошибок — используем Precision/Recall/F1, возможно с оптимизацией порога по целевой функции (например, максимум F1 или максимум Youden's J: $J = TPR - FPR$).

Дальше:
- Построим сводную таблицу метрик для всех обученных моделей и сравним их по ROC AUC и PR AUC (отдельными графиками, по одному графику в ячейке).
- Визуализируем ROC/PR кривые для трёх лучших моделей, чтобы увидеть различия в поведении по всему диапазону порогов.

In [8]:
# Сводная таблица метрик для всех моделей (Accuracy, F1, ROC AUC, PR AUC)
import numpy as np
import pandas as pd
from sklearn.metrics import accuracy_score, f1_score, roc_auc_score, average_precision_score

# Объединим модели
all_models = {}
try:
    all_models.update(models)
except NameError:
    all_models = {}
try:
    all_models.update(svc_models)
except NameError:
    pass

if not all_models:
    print("Не найдены обученные модели. Сначала выполните ячейки обучения.")
else:
    y_true = (y_test == 1).astype(int)
    rows = []
    probas_dict = {}
    for name, clf in all_models.items():
        # вероятности/скор
        if hasattr(clf, 'predict_proba'):
            y_proba = clf.predict_proba(X_test)[:, 1]
        elif hasattr(clf, 'decision_function'):
            z = clf.decision_function(X_test)
            y_proba = 1/(1+np.exp(-z))
        else:
            # как запасной вариант
            y_pred = clf.predict(X_test)
            y_proba = (y_pred == 1).astype(float)
        probas_dict[name] = y_proba
        y_pred = (y_proba >= 0.5).astype(int)
        rows.append({
            'model': name,
            'accuracy': accuracy_score(y_true, y_pred),
            'f1': f1_score(y_true, y_pred),
            'roc_auc': roc_auc_score(y_true, y_proba),
            'pr_auc': average_precision_score(y_true, y_proba)
        })
    metrics_df = pd.DataFrame(rows).sort_values('roc_auc', ascending=False)
    metrics_df

In [9]:
# Сравнение ROC AUC по моделям (столбчатая диаграмма)
import plotly.graph_objects as go
import pandas as pd

if 'metrics_df' not in globals():
    print("Сначала выполните ячейку с расчётом metrics_df (таблица метрик).")
else:
    fig = go.Figure(data=[
        go.Bar(name='ROC AUC', x=metrics_df['model'], y=metrics_df['roc_auc'], marker_color='#F58518')
    ])
    fig.update_layout(title='Сравнение моделей по ROC AUC', yaxis_title='ROC AUC')
    fig.show()

In [10]:
# Сравнение PR AUC по моделям (столбчатая диаграмма)
import plotly.graph_objects as go
import pandas as pd

if 'metrics_df' not in globals():
    print("Сначала выполните ячейку с расчётом metrics_df (таблица метрик).")
else:
    fig = go.Figure(data=[
        go.Bar(name='PR AUC', x=metrics_df['model'], y=metrics_df['pr_auc'], marker_color='#9467bd')
    ])
    fig.update_layout(title='Сравнение моделей по PR AUC', yaxis_title='PR AUC')
    fig.show()

In [11]:
# ROC-кривые для топ-3 моделей по ROC AUC
import numpy as np
import plotly.graph_objects as go
from sklearn.metrics import roc_curve

if 'metrics_df' not in globals():
    print("Сначала выполните ячейку с расчётом metrics_df (таблица метрик).")
else:
    top3 = list(metrics_df.head(3)['model'])
    y_true = (y_test == 1).astype(int)
    fig = go.Figure()
    for name in top3:
        if name in probas_dict:
            y_proba = probas_dict[name]
            fpr, tpr, _ = roc_curve(y_true, y_proba)
            fig.add_trace(go.Scatter(x=fpr, y=tpr, mode='lines', name=name))
    fig.update_layout(title='ROC-кривые для топ-3 моделей', xaxis_title='FPR', yaxis_title='TPR')
    fig.show()

In [12]:
# PR-кривые для топ-3 моделей по ROC AUC
import numpy as np
import plotly.graph_objects as go
from sklearn.metrics import precision_recall_curve

if 'metrics_df' not in globals():
    print("Сначала выполните ячейку с расчётом metrics_df (таблица метрик).")
else:
    top3 = list(metrics_df.head(3)['model'])
    y_true = (y_test == 1).astype(int)
    fig = go.Figure()
    for name in top3:
        if name in probas_dict:
            y_proba = probas_dict[name]
            prec, rec, _ = precision_recall_curve(y_true, y_proba)
            fig.add_trace(go.Scatter(x=rec, y=prec, mode='lines', name=name))
    fig.update_layout(title='PR-кривые для топ-3 моделей', xaxis_title='Recall', yaxis_title='Precision')
    fig.show()

## Важность признаков: теория и оценка для нашей задачи

- Деревья/ансамбли деревьев (RandomForest, Gradient Boosting, XGBoost/LightGBM/CatBoost) предоставляют относительные важности признаков, основанные на снижении нечистоты (Gini/Entropy) или приросте качества при сплитах. Эти важности относятся к обученной выборке и не равны причинному влиянию.
- Линейные модели (LinearSVC, SVC с linear kernel) имеют вектора коэффициентов w. При корректном масштабировании признаков модуль коэффициента |w_j| отражает вклад признака j в линейную границу. Сравнение коэффициентов корректно только при одинаковом масштабе фич.
- Kernel SVM (например, RBF) не имеет естественной «важности признаков» в исходном пространстве — полезнее применять пермутационную важность или SHAP.
- Пермутационная важность — модель-агностичный способ оценить вклад признака через падение метрики при случайной перестановке значений данного признака. Она устойчива к разным моделям, но требует доп. вычислений и кросс-валидации.

Практика на нашем 2D-датасете:
- Мы покажем тепловую карту важностей для моделей, где они доступны (feature_importances_ для деревьев, |coef_| для линейных SVM). Для RBF SVM важности не определены.
- Интерпретация: более тёмный цвет означает больший вклад признака (x1 или x2) в решающую функцию модели.
- Предостережения: при сильной корреляции фич важности могут «распределяться» между ними; сравнение различных алгоритмов требует нормировки важностей (мы нормируем суммы до 1).

In [13]:
# Оценка важности признаков: тепловая карта по доступным моделям
import numpy as np
import pandas as pd
import plotly.graph_objects as go

feature_names = ["x1", "x2"]
importance_map = {}

# Соберём модели, у которых есть важности/коэффициенты
all_models = {}
try:
    all_models.update(models)
except NameError:
    all_models = {}
try:
    all_models.update(svc_models)
except NameError:
    pass

selected = []
for name in ["RandomForest", "SklearnGB", "XGBoost", "LightGBM", "CatBoost", "SVC-linear", "LinearSVC"]:
    if name in all_models:
        selected.append(name)

for name in selected:
    m = all_models[name]
    imp = None
    if hasattr(m, "feature_importances_"):
        imp = np.asarray(m.feature_importances_).ravel()
    elif hasattr(m, "coef_"):
        imp = np.abs(np.asarray(m.coef_).ravel())
    if imp is not None:
        # Приведём к сумме 1 для сопоставимости
        s = imp.sum()
        if s > 0:
            imp = imp / s
        # Обрежем/дополнем до 2 признаков на всякий случай
        if imp.shape[0] >= 2:
            imp = imp[:2]
        else:
            imp = np.pad(imp, (0, 2-imp.shape[0]), constant_values=0.0)
        importance_map[name] = imp

if not importance_map:
    print("Нет моделей с доступной оценкой важности признаков (feature_importances_ или coef_).")
else:
    df_imp = pd.DataFrame(importance_map, index=feature_names)
    fig = go.Figure(data=go.Heatmap(
        z=df_imp.values,
        x=list(df_imp.columns),
        y=list(df_imp.index),
        colorscale="Blues",
        colorbar_title="Важность"
    ))
    fig.update_layout(title="Важность признаков по моделям (нормировано)")
    fig.show()

## Плюсы и минусы методов, рекомендации по применению

Gradient Boosted Trees (XGBoost/LightGBM/CatBoost)
- Плюсы: высокая точность на табличных данных; устойчивость к разным масштабам признаков; встроенная обработка пропусков; гибкая регуляризация; важности признаков.
- Минусы: чувствительность к гиперпараметрам (n_estimators, learning_rate, max_depth); время обучения на очень больших датасетах; слабая интерпретируемость по сравнению с линейными моделями.
- Когда применять: табличные данные, нелинейные зависимости, ограниченные размеры фичей; лидирует во многих Kaggle-задачах.
- Когда не применять: очень высокоразмерные разреженные данные (тексты) — иногда линейные модели лучше по скорости/обобщению; когда нужна строгая интерпретация коэффициентов.

SVM (linear и kernel)
- Плюсы: теоретически обоснованная максимизация margin; хорош на средних размерах данных; kernel trick позволяет улавливать сложные нелинейности.
- Минусы: чувствительность к выбору C и ядра (и gamma); масштабируемость хуже для очень больших n (особенно kernel SVM); требуется масштабирование признаков для линейного SVM.
- Когда применять: размер данных умеренный; чёткая граница между классами; хорошо подготовленные признаки; при необходимости устойчивого решения с контролем margin.
- Когда не применять: крайне большие датасеты или количество признаков с плотной матрицей — обучение kernel SVM может быть слишком медленным; когда нужна очень быстрая инференс/обучение.

Random Forest
- Плюсы: устойчивый базовый алгоритм; мало гиперпараметров; неплохо работает из коробки; оценка важностей признаков.
- Минусы: часто проигрывает boosting по качеству; может переобучаться на шумных данных без тщательной настройки; медленнее на инференсе при большом количестве деревьев.
- Когда применять: быстрый сильный baseline; когда не хочется тонкой настройки; для начальной оценки сложности задачи.

Итого: на табличных данных стартуем с Random Forest как baseline, затем применяем Gradient Boosting. SVM применяем при умеренном размере данных и наличии хорошо масштабированных признаков; kernel SVM — когда есть ожидание сложной нелинейной границы и достаточно ресурсов.

## TO-DO

Цель: закрепить навыки работы с Gradient Boosted Trees и SVM, а также их сравнение с Random Forest.

1) Подготовка данных (10%)
- Загрузите или сгенерируйте собственный 2D набор для бинарной классификации (например, `make_circles` или реальный датасет). Разделите на train/test. Опишите кратко данные.

2) Gradient Boosted Trees (30%)
- Обучите две модели из семейства boosting (например, XGBoost и LightGBM или их аналог из sklearn). Подберите базовые гиперпараметры (n_estimators, learning_rate, max_depth). Сравните Accuracy и ROC AUC. Кратко интерпретируйте результат.

3) SVM: linear и kernel (30%)
- Обучите LinearSVC и SVC(RBF) на ваших данных. Исследуйте влияние гиперпараметров C и gamma (для RBF) с помощью краткого grid search или нескольких ручных запусков. Сравните с boosting и Random Forest.

4) Визуализация границ решений (20%)
- Постройте визуализацию границы решений для лучших по ROC AUC моделей в каждой группе: один boosting, один SVM. Используйте Plotly; убедитесь, что на графике видны точки train/test (разными символами/цветами).

5) Аналитика и выводы (10%)
- Сформулируйте, когда на вашем типе данных SVM выигрывает/проигрывает boosting и наоборот. Укажите, какие гиперпараметры оказались наиболее критичны.

Требования к оформлению:
- Комментарии к коду в формате NumPy Docstring.
- Результат оформить как Notebook, приложить краткий отчёт (markdown в том же Notebook).

## Чек-лист: типичные ошибки и анти‑паттерны

- Отсутствие масштабирования для SVM: `StandardScaler` обязателен для стабильной настройки `C` и `gamma`.
- Сравнение моделей на разных разбиениях: используйте одинаковый `train_test_split` или кросс-валидацию.
- Неправильная интерпретация ROC AUC при дисбалансе: при сильном дисбалансе смотрите PR AUC и изменяйте порог.
- Переобучение Boosting при большом `learning_rate`: снижайте `learning_rate`, повышайте `n_estimators`, используйте early stopping.
- Игнорирование стохастики: фиксируйте `random_state/SEED` для воспроизводимости, используйте `subsample/colsample` с умом.
- Сравнение метрик без уверенности в пороге: оценивайте кривые (ROC/PR), подбирайте порог по бизнес-критериям.
- Неверная трактовка feature importance: помните, что важности деревьев — относительные; проверяйте устойчивость на различных разбиениях.